روی دیتای

cardio_train1

شبکه عمیق خود را به صورت های

sequentional functional classbase

و تیون کردن هایپر پارمتر ها ترین کنید

1- حتما تنسور بورد داشته باشد و به صورت یک لینک داده شود

2-EarlyStopping

3-  در ابتدای هر ایپاک شماره ایپاک و مقدار لاس را در متن زیر پرینت کند

in ... epoch loss is ...


In [32]:
import pandas as pd
import numpy as np
import datetime

In [8]:
df= pd.read_csv('cardio_train1.csv',index_col='id')



In [12]:
df.isnull().values.any()

False

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          70000 non-null  int64  
 1   gender       70000 non-null  int64  
 2   height       70000 non-null  int64  
 3   weight       70000 non-null  float64
 4   ap_hi        70000 non-null  int64  
 5   ap_lo        70000 non-null  int64  
 6   cholesterol  70000 non-null  int64  
 7   gluc         70000 non-null  int64  
 8   smoke        70000 non-null  int64  
 9   alco         70000 non-null  int64  
 10  active       70000 non-null  int64  
 11  cardio       70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.9 MB


In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X= df.iloc[:,:-1].values
Y=df.iloc[:,-1].values
X=scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.33, random_state=42)
X_val, X_test, y_val, y_test = train_test_split( X_test, y_test, test_size=0.33, random_state=42)





In [28]:
import tensorflow as tf 



In [30]:
tf.random.set_seed(42)
model= tf.keras.Sequential([tf.keras.layers.Dense(300,activation='relu'),tf.keras.layers.Dense(150,activation='relu'),tf.keras.layers.Dense(2,activation='softmax')])

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-03 18:25:33.288265: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-03 18:25:33.288773: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [37]:
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(
            "The  loss for epoch {} is {:7.2f} ".format(
                epoch, logs["loss"]
            ))

In [39]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.fit(X_train,y_train,epochs=10,validation_data=(X_val,y_val), callbacks=[callback,LossAndErrorPrintingCallback()])

Epoch 1/10


2023-03-03 18:46:50.833394: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1466/1466 [==============================] - ETA: 0s - loss: 0.6321 - accuracy: 0.6441

2023-03-03 18:46:58.875122: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


The  loss for epoch 0 is    0.63 
1466/1466 [==============================] - 10s 6ms/step - loss: 0.6321 - accuracy: 0.6441 - val_loss: 0.6191 - val_accuracy: 0.6537
Epoch 2/10
1466/1466 [==============================] - 9s 6ms/step - loss: 0.6186 - accuracy: 0.6586 - val_loss: 0.6092 - val_accuracy: 0.6663
Epoch 3/10
1466/1466 [==============================] - 9s 6ms/step - loss: 0.6082 - accuracy: 0.6695 - val_loss: 0.5991 - val_accuracy: 0.6802
Epoch 4/10
1466/1466 [==============================] - 9s 6ms/step - loss: 0.5959 - accuracy: 0.6880 - val_loss: 0.5842 - val_accuracy: 0.7020
Epoch 5/10
1466/1466 [==============================] - 9s 6ms/step - loss: 0.5823 - accuracy: 0.7065 - val_loss: 0.5709 - val_accuracy: 0.7137
Epoch 6/10
1466/1466 [==============================] - 9s 6ms/step - loss: 0.5721 - accuracy: 0.7182 - val_loss: 0.5642 - val_accuracy: 0.7227
Epoch 7/10
1466/1466 [==============================] - 9s 6ms/step - loss: 0.5663 - accuracy: 0.7213 - val_loss

In [41]:
normaization_layer = tf.keras.layers.Normalization()
hidden_1=tf.keras.layers.Dense(300,activation='relu')
hidden_2=tf.keras.layers.Dense(250, activation='relu')
concat_layer= tf.keras.layers.Concatenate()
output_layer= tf.keras.layers.Dense(10,activation='softmax')
input_=tf.keras.layers.Input(shape=X_train.shape[1])
normalized = normaization_layer(input_)
hidden1=hidden_1(normalized)
hidden2=hidden_2(hidden1)
concat=concat_layer([hidden1,hidden2])
output=output_layer(concat)
model=tf.keras.Model(inputs=[input_],outputs=[output])
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=30,validation_data=(X_val,y_val),callbacks=[callback,LossAndErrorPrintingCallback()])

Epoch 1/30
   1/1466 [..............................] - ETA: 7:31 - loss: 2.4016 - accuracy: 0.0000e+00

2023-03-03 18:51:42.445332: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1466/1466 [==============================] - ETA: 0s - loss: 0.7054 - accuracy: 0.6413

2023-03-03 18:51:50.900362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


The  loss for epoch 0 is    0.71 
1466/1466 [==============================] - 10s 7ms/step - loss: 0.7054 - accuracy: 0.6413 - val_loss: 0.6214 - val_accuracy: 0.6559
Epoch 2/30
1466/1466 [==============================] - 10s 7ms/step - loss: 0.6202 - accuracy: 0.6608 - val_loss: 0.6074 - val_accuracy: 0.6735
Epoch 3/30
1466/1466 [==============================] - 10s 7ms/step - loss: 0.6077 - accuracy: 0.6759 - val_loss: 0.5960 - val_accuracy: 0.6907
Epoch 4/30
1466/1466 [==============================] - 10s 7ms/step - loss: 0.5964 - accuracy: 0.6954 - val_loss: 0.5841 - val_accuracy: 0.7052
Epoch 5/30
1466/1466 [==============================] - 10s 7ms/step - loss: 0.5864 - accuracy: 0.7065 - val_loss: 0.5727 - val_accuracy: 0.7155
Epoch 6/30
1466/1466 [==============================] - 10s 7ms/step - loss: 0.5790 - accuracy: 0.7153 - val_loss: 0.5686 - val_accuracy: 0.7209
Epoch 7/30
1466/1466 [==============================] - 10s 7ms/step - loss: 0.5743 - accuracy: 0.7188 - va

linux

1- با چه دستوری مسیر پایتون را پیدا کنیم



2- با چه دستوری از نوع فایل مطمین میشبم



3-منظور از اعداد نشان داده شده بعد از زدت دستور زیر چیست



4- دستوری که دایرکتوری بسازد که درون آن یک دایرکتوری دیگر هم هست



5- با چه دستوری فایل بالا را به صورت ریکرسیو پاک می کنید



6- با چه کامتد هایی میتوان فایل را خواند 



7- برای تعقیب تعییرات یک فایل چه دستوری را پیشنهاد میدهید


8- معادل دستورپرینت در بش چیست 

